### Import Library

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
nltk.download('stopwords')

nltk.download('all')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Acer\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\Acer\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\Acer\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\Acer\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     C:\Users\Acer\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |

## Data Preparation


In [2]:
# Baca masing-masing file
df1 = pd.read_csv("Phishing_Email.csv")
df2 = pd.read_csv("deceptive-opinion.csv")
df3 = pd.read_csv("proc_email.csv")

# Gabungkan jadi satu dataframe
df = pd.concat([df1, df2, df3], ignore_index=True)

In [3]:
df1.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18650 entries, 0 to 18649
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  18650 non-null  int64 
 1   Email Text  18634 non-null  object
 2   Email Type  18650 non-null  object
dtypes: int64(1), object(2)
memory usage: 437.2+ KB


In [4]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600 entries, 0 to 1599
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   deceptive  1600 non-null   object
 1   hotel      1600 non-null   object
 2   polarity   1600 non-null   object
 3   source     1600 non-null   object
 4   text       1600 non-null   object
dtypes: object(5)
memory usage: 62.6+ KB


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33758 entries, 0 to 33757
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  18650 non-null  float64
 1   Email Text  18634 non-null  object 
 2   Email Type  18650 non-null  object 
 3   deceptive   1600 non-null   object 
 4   hotel       1600 non-null   object 
 5   polarity    1600 non-null   object 
 6   source      1600 non-null   object 
 7   text        1600 non-null   object 
 8   To          13508 non-null  object 
 9   From        13508 non-null  object 
 10  X-To        13508 non-null  object 
 11  X-From      13508 non-null  object 
 12  content     13508 non-null  object 
 13  to_index    13508 non-null  object 
dtypes: float64(1), object(13)
memory usage: 3.6+ MB


In [6]:
print(df1["Email Type"].unique())

['Safe Email' 'Phishing Email']


## Data Preprocessing


In [7]:
df_phising = df1[["Email Text", "Email Type"]]
df_phising.head()

,Email Text,Email Type
0,"re : 6 . 1100 , disc : uniformitarianism , re ...",Safe Email
1,the other side of * galicismos * * galicismo *...,Safe Email
2,re : equistar deal tickets are you still avail...,Safe Email
3,\nHello I am your hot lil horny toy.\n I am...,Phishing Email
4,software at incredibly low prices ( 86 % lower...,Phishing Email


In [8]:
df_phising = df_phising.rename(columns={"Email Text": "text", "Email Type": "label"})
df_phising.head()

,text,label
0,"re : 6 . 1100 , disc : uniformitarianism , re ...",Safe Email
1,the other side of * galicismos * * galicismo *...,Safe Email
2,re : equistar deal tickets are you still avail...,Safe Email
3,\nHello I am your hot lil horny toy.\n I am...,Phishing Email
4,software at incredibly low prices ( 86 % lower...,Phishing Email


In [9]:
df_phising["label"] = df_phising["label"].map({
    "Safe Email": 0,        # Asli
    "Phishing Email": 2     # Penipuan/Phishing
})

In [10]:
df_deceptive = df2[["text", "deceptive"]]
df_deceptive.head()

,text,deceptive
0,We stayed for a one night getaway with family ...,truthful
1,Triple A rate with upgrade to view room was le...,truthful
2,This comes a little late as I'm finally catchi...,truthful
3,The Omni Chicago really delivers on all fronts...,truthful
4,I asked for a high floor away from the elevato...,truthful


In [11]:
df_deceptive = df_deceptive.rename(columns={"deceptive": "label"})
df_deceptive.head()

,text,label
0,We stayed for a one night getaway with family ...,truthful
1,Triple A rate with upgrade to view room was le...,truthful
2,This comes a little late as I'm finally catchi...,truthful
3,The Omni Chicago really delivers on all fronts...,truthful
4,I asked for a high floor away from the elevato...,truthful


In [12]:
df_deceptive["label"] = df_deceptive["label"].map({
    "truthful": 0,
    "deceptive": 2
})

In [13]:
df_proc = df3[["content"]].rename(columns = {"content": "text"})
df_proc.head()

,text
0,ENA Contact\n\nDaren Farmer\nPhone # 713-853-6...
1,Agreed - can you please confirm directly with ...
2,Please send Pam Benson the information for Joe...
3,Blue Range Resources Corporation is an execute...
4,"Andrew, Attached is the CSA. I added some la..."


In [14]:
df_proc["label"] = None

In [15]:
df_supervised = pd.concat([df_phising, df_deceptive], ignore_index=True)
df_unlabeled = df_proc.copy()

In [16]:
print(df_supervised)

                                                    text  label
0      re : 6 . 1100 , disc : uniformitarianism , re ...      0
1      the other side of * galicismos * * galicismo *...      0
2      re : equistar deal tickets are you still avail...      0
3      \nHello I am your hot lil horny toy.\n    I am...      2
4      software at incredibly low prices ( 86 % lower...      2
...                                                  ...    ...
20245  Problems started when I booked the InterContin...      2
20246  The Amalfi Hotel has a beautiful website and i...      2
20247  The Intercontinental Chicago Magnificent Mile ...      2
20248  The Palmer House Hilton, while it looks good i...      2
20249  As a former Chicagoan, I'm appalled at the Ama...      2

[20250 rows x 2 columns]


In [17]:
print(df_unlabeled)

                                                    text label
0      ENA Contact\n\nDaren Farmer\nPhone # 713-853-6...  None
1      Agreed - can you please confirm directly with ...  None
2      Please send Pam Benson the information for Joe...  None
3      Blue Range Resources Corporation is an execute...  None
4      Andrew,  Attached is the CSA.  I added some la...  None
...                                                  ...   ...
13503  Second message from Bruce.\n------------------...  None
13504  What occurred at this point in prior months?  ...  None
13505  We are half way home - You passed the drug tes...  None
13506  please print\n----- Forwarded by Steven J Kean...  None
13507  Do you have a copy of Maureen's deal ticket?\n...  None

[13508 rows x 2 columns]


In [18]:
df_supervised.isnull().sum()

text     16
label     0
dtype: int64

In [19]:
print(df_supervised["label"].value_counts())

label
0    12122
2     8128
Name: count, dtype: int64


## StopWord, Tokenizer, Stemming


In [20]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

# Inisialisasi tools NLP
nltk.download('punkt')
nltk.download('stopwords')
ps = PorterStemmer()

# Load slang dictionary aman
slang_dict = {}
with open("slang.txt", "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line or line.startswith("#"):
            continue
        if "=" in line:
            key, value = line.split("=", 1)
        elif ":" in line:
            key, value = line.split(":", 1)
        else:
            continue
        slang_dict[key.strip().lower()] = value.strip().lower()

# Fungsi formalization slangword
def convertToSlangword(tokens):
    return [slang_dict.get(word.lower(), word.lower()) for word in tokens]

# Fungsi cleaning
def cleaning_email(isi):
    isi = re.sub(r'@[A-Za-z0-9]+',' ',isi)  # hapus mention
    isi = re.sub(r'#[A-Za-z0-9]+',' ',isi)  # hapus hashtag
    isi = re.sub(r"http\S+",' ',isi)        # hapus URL
    isi = re.sub(r'[0-9]+',' ',isi)         # hapus angka
    isi = re.sub(r"[-()\"#/@;:<>{}'+=~|.!?,_]", " ", isi)  # hapus simbol
    return isi.strip()

# Fungsi utama preprocessing
def preprocess_dataframe(df, text_col="text"):
    df[text_col] = df[text_col].fillna("").astype(str)
    # 1. Cleaning dasar
    df['cleaning'] = df[text_col].apply(cleaning_email)

    # 2. Hapus emoji
    df['hapusEmoji'] = df['cleaning'].apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))

    # 3. Replace triple or more character
    df['replaceTOM'] = df['hapusEmoji'].apply(lambda x: re.sub(r'(.)\1{2,}', r'\1', x))

    # 4. Case folding
    df['caseFolding'] = df['replaceTOM'].str.lower()

    # 5. Tokenizing
    df['tokenizing'] = df['caseFolding'].apply(word_tokenize)

    # 6. Formalisasi slangword
    df['formalisasi'] = df['tokenizing'].apply(convertToSlangword)

    # 7. Stopword removal
    stopword_list = set(stopwords.words('english'))
    stopword_list.update(["subject", "re", "fw", "fwd", "etc", "ok", "thank", "thanks", "hi", "hello", "regards", "dear"])
    df['removeStop'] = df['formalisasi'].apply(lambda tokens: [w for w in tokens if w not in stopword_list])

    # 8. Stemming
    df['stemming'] = df['removeStop'].apply(lambda tokens: [ps.stem(w) for w in tokens])

    # 9. Join jadi final text
    df['final_text'] = df['stemming'].apply(lambda tokens: " ".join(tokens))

    return df


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Acer\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Acer\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
# =========================
# 1. Preprocess dataset
# =========================
df_supervised = preprocess_dataframe(df_supervised, text_col="text")
df_unlabeled = preprocess_dataframe(df_unlabeled, text_col="text")

# =========================
# 2. Lihat hasil
# =========================
print(df_supervised[['text', 'label', 'final_text']].head())
print(df_unlabeled[['text', 'final_text']].head())

# =========================
# 3. Cek distribusi label
# =========================
print(df_supervised['label'].value_counts())


                                                text  label  \
0  re : 6 . 1100 , disc : uniformitarianism , re ...      0   
1  the other side of * galicismos * * galicismo *...      0   
2  re : equistar deal tickets are you still avail...      0   
3  \nHello I am your hot lil horny toy.\n    I am...      2   
4  software at incredibly low prices ( 86 % lower...      2   

                                          final_text  
0  disc uniformitarian sex lang dick hudson obser...  
1  side * galicismo * * galicismo * spanish term ...  
2  equistar deal ticket still avail assist robert...  
3  hot lil horni toy one dream open mind person l...  
4  softwar incred low price % lower draperi seven...  
                                                text  \
0  ENA Contact\n\nDaren Farmer\nPhone # 713-853-6...   
1  Agreed - can you please confirm directly with ...   
2  Please send Pam Benson the information for Joe...   
3  Blue Range Resources Corporation is an execute...   
4  Andrew, 

## Split Data

In [22]:
from sklearn.model_selection import train_test_split

X = df_supervised['final_text']
y = df_supervised['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=10000, ngram_range=(1,2))

In [24]:
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

## Modeling

In [25]:
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.linear_model import LogisticRegression
# from sklearn.svm import LinearSVC
# from sklearn.model_selection import cross_val_score
# from sklearn.metrics import accuracy_score, classification_report

# # Naive Bayes
# model_nb = MultinomialNB()
# model_nb.fit(X_train_tfidf, y_train)
# pred_nb = model_nb.predict(X_test_tfidf)
# acc_nb = cross_val_score(model_nb, X_train_tfidf, y_train, cv=3, scoring='accuracy')

# # SVM
# model_svm = LinearSVC()
# model_svm.fit(X_train_tfidf, y_train)
# pred_svm = model_svm.predict(X_test_tfidf)

# # Logistic Linear
# model_lr = LogisticRegression()
# model_lr.fit(X_train_tfidf, y_train)
# pred_lr = model_lr.predict(X_test_tfidf)

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np

models = {
    "Naive Bayes": MultinomialNB(),
    "Logistic Regression": LogisticRegression(max_iter=1000, solver="liblinear"),
    "SVM": LinearSVC()
}

for name, model in models.items():
    print(f"\n===== {name} =====")
    # Train
    model.fit(X_train_tfidf, y_train)

    # Prediction
    preds = model.predict(X_test_tfidf)

    # Accuracy
    acc = accuracy_score(y_test, preds)
    print("Accuracy:", acc)

    # Classification report
    print("Classification Report:\n", classification_report(y_test, preds))

    # Confusion matrix
    print("Confusion Matrix:\n", confusion_matrix(y_test, preds))

    # Cross-validation
    cv_scores = cross_val_score(model, X_train_tfidf, y_train, cv=3, scoring="accuracy")
    print("Cross-val scores:", cv_scores)
    print("Mean accuracy:", np.mean(cv_scores))





===== Naive Bayes =====
Accuracy: 0.9175308641975308
Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.90      0.93      2403
           2       0.87      0.94      0.90      1647

    accuracy                           0.92      4050
   macro avg       0.91      0.92      0.92      4050
weighted avg       0.92      0.92      0.92      4050

Confusion Matrix:
 [[2167  236]
 [  98 1549]]
Cross-val scores: [0.93925926 0.92425926 0.92685185]
Mean accuracy: 0.9301234567901234

===== Logistic Regression =====
Accuracy: 0.9419753086419753
Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.95      0.95      2403
           2       0.93      0.93      0.93      1647

    accuracy                           0.94      4050
   macro avg       0.94      0.94      0.94      4050
weighted avg       0.94      0.94      0.94      4050

Confusion Matrix:
 [[2282  121]
 [ 114 1533]]
Cro

## Inference

In [26]:
# Mapping label ke kategori
label_map = {0: "ASLI", 2: "PHISHING"}

# Contoh email
email_asli = '''Subject: Security alert: A new sign-in to your account

From: My Services Corp Security no-reply@account.myservicescorp.com

To: john.doe@email.com

Hello John Doe,

Your My Services Corp account was just used to sign in from a new browser. We are sending you this notification to ensure your account remains secure.

Sign-in details:

Time: Monday, September 22, 2025, 5:10 PM (WIB)

Device: Chrome on Windows

Location: Jakarta, Indonesia (approximate)

If this was you, you can safely ignore this email. Your device has been added to your trusted devices list.

If this was not you, your account may have been compromised. Please follow these steps immediately:

Click here to review your account activity and secure your account:
Review Activity

You will be guided to change your password and review your security settings.

Worried about this email's authenticity?
Do not click any links. Instead, open your web browser and manually type https://account.myservicescorp.com into the address bar. From there, you can sign in and check your account's recent activity.

Thanks,
The My Services Corp Team

© 2025 My Services Corp Inc., 123 Main Street, Anytown, USA.
This is a mandatory service email to inform you of important changes to your account.'''
email_phish = '''Subject: Urgent: Unusual Sign-In Activity on Your Account - Action Required

From: Security Support no-reply@cld-security-update.com

To: recipient.email@example.com

Dear Valued Customer,

We detected an unusual sign-in attempt to your account from a location we don't recognize.

Details:

Time: 22 September 2025, 03:15 AM (UTC)

Location: Moscow, Russia (IP Address: 103.77.21.89)

Device: Unknown Browser on Windows 10

For your protection, we have temporarily suspended your account. To regain access, you must verify your identity and confirm your recent activity immediately.

Please click the link below to review the activity and secure your account.

SECURE YOUR ACCOUNT NOW

If you do not take action within the next 12 hours, your account will be permanently disabled to prevent further unauthorized access.

If this was you, you can disregard this email. However, we strongly recommend reviewing your security settings.

Thank you for your cooperation.

Sincerely,
The Cloud Services Security Team

'''

# Pilih model terbaik (misal: SVM)
best_model = models["SVM"]

# Preprocessing manual supaya sama dengan pipeline
def preprocess_single_text(text):
    text = cleaning_email(text)
    text = text.encode('ascii', 'ignore').decode('ascii')
    text = re.sub(r'(.)\1{2,}', r'\1', text)
    text = text.lower()
    tokens = word_tokenize(text)
    tokens = convertToSlangword(tokens)
    stopword_list = set(stopwords.words('english'))
    stopword_list.update(["subject","re","fw","fwd","etc","ok","thank","thanks","hi","hello","regards","dear"])
    tokens = [w for w in tokens if w not in stopword_list]
    tokens = [ps.stem(w) for w in tokens]
    return " ".join(tokens)

# Preprocess & transform
emails = [email_asli, email_phish]
emails_processed = [preprocess_single_text(e) for e in emails]
emails_tfidf = tfidf.transform(emails_processed)

# Predict
preds = best_model.predict(emails_tfidf)

# Print hasil
for i, email in enumerate(emails):
    print("\nEmail:", email)
    print("Prediksi:", label_map.get(preds[i], preds[i]))


Email: Subject: Security alert: A new sign-in to your account

From: My Services Corp Security no-reply@account.myservicescorp.com

To: john.doe@email.com

Hello John Doe,

Your My Services Corp account was just used to sign in from a new browser. We are sending you this notification to ensure your account remains secure.

Sign-in details:

Time: Monday, September 22, 2025, 5:10 PM (WIB)

Device: Chrome on Windows

Location: Jakarta, Indonesia (approximate)

If this was you, you can safely ignore this email. Your device has been added to your trusted devices list.

If this was not you, your account may have been compromised. Please follow these steps immediately:

Click here to review your account activity and secure your account:
Review Activity

You will be guided to change your password and review your security settings.

Worried about this email's authenticity?
Do not click any links. Instead, open your web browser and manually type https://account.myservicescorp.com into the addr

In [ ]:
import pickle

# Simpan TF-IDF vectorizer
with open('tfidf.pkl', 'wb') as f:
    pickle.dump(tfidf, f)

# Simpan model terbaik (misal SVM)
with open('model.pkl', 'wb') as f:
    pickle.dump(best_model, f)

print("✅ Model dan TF-IDF berhasil disimpan!")

✅ Model dan vectorizer berhasil disimpan:
/models/saved_models\svm_phishing_model.pkl
/models/saved_models\tfidf_vectorizer.pkl
